<h1> Intercambio de Llaves Diffie Hellman y El Gamal

Los siguientes cifrados, son alternativas al cifrado RSA para el intercambio de Llaves. La seguridad de ambos cifrados
radica principalmente en el dificil cálculo del logaritmo discreto.

El primero, el cifrado de Diffie - Hellman, fue establecido por Whitfield Diffie y Martin Hellman. Consiste en lo siguiente: ($Z_p$)
         
1. Alicia escoge un grupo $Z_p$ y un generador (g), igualmente escoge una llave privada a. 
2. Alicia cálcula $A\ \equiv \ g^a\ mod\ p$ y le manda A a Bob.
3. Bob teniendo su llave privada (b) y teniendo g y $Z_p$, los cuales son públicos, cálcula $\ B \equiv g^b\ mod\ p$.
4. Bob le manda B a alicia.
5. Alicia hace $B^a=g^{ab}$ y bob hace $A^b=g^{ab}$.
6. Ambos obtienen la misma llave.

<h2> Ejemplo: (Intercambiemos Llaves)


Alicia escoge $g=13$ en $Z_{37124508045065437}$.

Escoge su llave privada $a=12736871092846$

In [2]:
a=12736871092846
g=13
p=37124508045065437
print ("a =",a)
print ("g =",g)
print ("p =",p)

a = 12736871092846
g = 13
p = 37124508045065437


Alicia calcula $\ A \equiv \ g^a\ mod\ p$ y se lo manda a Bob.

In [3]:
A=pow(g,a,p)
print ("A =",A)

A = 30384510001166445


Bob escoge su llave privada $b=8763247901223$, calcula $\ B \equiv \ g^b\ mod\ p$ y se lo manda a Alicia

In [4]:
b=87632478723934
B=pow(g,b,p)
print ("B =",B)

B = 20299696698362165


Alicia calcula $\ B^a\ mod\ p$ y Bob calcula $\ A^b\ mod\ p$.


In [5]:
llave_alicia=pow(B,a,p)
llave_bob=pow(A,b,p)
print ("Llave Alicia =",llave_alicia)
print ("Llave Bob =",llave_bob)

Llave Alicia = 4908577199504481
Llave Bob = 4908577199504481


Ambos consiguen la misma llave y pueden empezar a encriptar con llave privada.


<h2> El Gamal

Para el segundo cifrado para intercambio de llaves el proceso es un poco más complejo. El cifrado el Gamal a diferencia del cifrado Diffie Hellman es asimétrico, lo cual incrementa su grado de complejidad, además de que este cifrado puede igualmente ser utilizado para el cifrado de texto. Consiste en lo siguiente:

1. Alicia al igual que en Diffie Hellman escoge g, p, a(su llave aleatoria privada), y calcula $\ A \equiv \ g^a\ mod\ p$.
2. Hace públicas g y p y le manda A a Bob.
3. Ahora bob escoge un número K al y un texto (M) que pertenezca al grupo.
4. Bob va a calcular dos variables extras ($c_1$ y $c_2$) de la siguiente manera:
      $\ c_1 \equiv \ g^K\ mod\ p$  y 
      $\ c_2 = M*A^K$
5. Bob envía a Alicia $c_1$ y $c_2$.
6. Alicia define $\ x \equiv c_1^a\ mod\ p$.
7. Alicia calcula $\ x^{-1}*c_2 \equiv c_1^{-a}*M*A^K \equiv g^{-ka}*M*g^{ak} \equiv \ M\ mod\ p$
     

<h3> Ejemplo (intercambio de texto)

Alicia escoge igual que en el ejemplo anterior g = 13, p=633910111 y a distinto a = 71682736492998029374, y calcula A

In [111]:
a=71682723
g=13
p=179424673
A=pow(g,a,p)
print ("a =",a)
print ("g =",g)
print ("p =",p)
print ("A =",A)

a = 71682723
g = 13
p = 3367900312
A = 2623801389


Bob escoge K = 9821640917248672790347910 y calcula c1 y c2. Igualmente escoge el texto M = 546780193. Recibe A de Alicia. Bob le manda c1 y c2 a Alicia.

In [112]:
K=9821640917248672790347910
M=546780193
c1=pow(g,K,p)
c2=M*pow(A,K,p)
print ("Texto =",M)
print ("c1 =",c1)
print ("c2 =",c2)

Texto = 546780193
c1 = 2849326481
c2 = 915680108837644137


Alicia define X = c1^a mod p y calcula X^(-1)*c2

In [50]:
X=pow(c1,a,p)
def extended_gcd(aa, bb):
    lastremainder, remainder = abs(aa), abs(bb)
    x, lastx, y, lasty = 0, 1, 1, 0
    while remainder:
        lastremainder, (quotient, remainder) = remainder, divmod(lastremainder, remainder)
        x, lastx = lastx - quotient*x, x
        y, lasty = lasty - quotient*y, y
    return lastremainder, lastx * (-1 if aa < 0 else 1), lasty * (-1 if bb < 0 else 1)
def modinv(a, m):
    g, x, y = extended_gcd(a, m)
    if g != 1:
        raise ValueError
    return x % m
Xinv=modinv(X,p)
texto_decifrado=(Xinv*c2)%p
print("Texto Descifrado =",texto_decifrado)

Texto Descifrado = 546780193


<h1> Logaritmo Discreto


El logaritmo discreto, es la base de la seguridad de los metodos anteriores. Consiste en encontrar las llaves privadas de Bob y Alicia, a y b. Imaginemos que existe un tercer sujeto llamado Pancho. Pancho intercepta la comunicación entre Alicia y Bob y lo único que ve en el camino son A y B, es decir $g^a$ y $g^b$. Si Pancho pudiera saber a o b, entonces solo tomo ya sea A o B y lo eleva a la llave que encontro. Para encontrarlo necesita del logaritmo discreto.

El logaritmo discreto se define como:

$\ g^a \equiv \ A \ mod\ p$

Donde A, g y p son conocidos y buscamos a, es decir:

$ a \equiv \ log_g(A) \ mod\ p$

Hay varias formas de encontrar el logaritmo discreto. Tomemos del ejemplo anterior en el encriptado de Diffie Helman, los mismos números. 

El siguiente método se llama fuerza bruta, y simplemente consiste en elevar g a la x modulo p hasta conseguir A. Digamos que Pancho consigue A = 211508765, sabemos que g = 13 y p = 3367900313


In [125]:
A = 656334248
g = 13
p = 3367900313


    

<h3> Algoritmo Pohlig-Helman

In [123]:
from sympy.ntheory import factorint
factorint(3367900312)

{2: 3, 7: 1, 109: 1, 551753: 1}

Tomamos primero el factor $2^3$

In [126]:
int1=int((3367900313-1)/2)

alpha1=pow(13,int1,p)
print("Alpha1 =",alpha1)
beta1=pow(A,int1,p)
print("Beta1 =",beta1)


Alpha1 = 3367900312
Beta1 = 3367900312
1


Por lo tanto $x_0=1$, y ahora incrementamos $2^2$

In [136]:
s=modinv(g,p)
A1=A*s%p
print(A1)
int2=int((3367900312)/4)
beta2=pow(A1,int2,p)
print("Beta2 =",beta2)
prueba=pow(alpha1,5,p)
print("prueba =",prueba)

3159318308
Beta2 = 3367900312
prueba = 3367900312


Por lo tanto $x_1=1$

In [138]:
s2=modinv(g,p)
A2=A1*s2*s2%p
print(A2)
int3=int((3367900312)/8)
beta3=pow(A2,int3,p)
print("Beta3 =",beta3)


1732536954
Beta3 = 3367900312


Por lo tanto $x_2=1$, y la primera ecuación es: $x_1 = x_0+2*x_1+4*x_2 \equiv 7\ mod \ 8$

Tomamos primero el factor $7$

In [146]:
int11=int((3367900313-1)/7)

alpha11=pow(13,int11,p)
print("Alpha1 =",alpha11)
beta11=pow(A,int11,p)
print("Beta1 =",beta11)

Alpha1 = 2974477553
Beta1 = 1


Por lo tanto  $x_1=0$ y la segunda ecuación es: $x_2 \equiv 0\ mod \ 7$

Tomamos primero el factor $7$

In [147]:
int111=int((3367900313-1)/109)

alpha111=pow(13,int111,p)
print("Alpha1 =",alpha111)
beta111=pow(A,int111,p)
print("Beta1 =",beta111)

Alpha1 = 2920505388
Beta1 = 2497218448


In [149]:
prueba=pow(alpha111,3,p)
print (prueba)

2497218448


Por lo tanto  $x_1=3$ y la segunda ecuación es: $x_3 \equiv 3\ mod \ 109$


Tomamos primero el factor $7$

In [150]:
int1111=int((3367900313-1)/551753)

alpha1111=pow(13,int1111,p)
print("Alpha1 =",alpha1111)
beta1111=pow(A,int1111,p)
print("Beta1 =",beta1111)

Alpha1 = 600054388
Beta1 = 2491247764


In [159]:
i=0
result=0
while result != beta1111:
    result=pow(alpha1111,i,p)
    i=i+1
print(i-1)
print (result)

46552
2491247764


In [160]:
print(pow(alpha1111,46552,p))

2491247764


Por lo tanto  $x_1=46552$ y la segunda ecuación es: $x_4 \equiv 46552\ mod \ 551753$

In [163]:
def chinese_remainder(n, a):
    sum = 0
    prod=reduce(lambda a, b: a*b, n)
 
    for n_i, a_i in zip(n, a):
        p = prod / n_i
        sum += a_i * mul_inv(p, n_i) * p
    return sum % prod
 
 
def mul_inv(a, b):
    b0 = b
    x0, x1 = 0, 1
    if b == 1: return 1
    while a > 1:
        q = a / b
        a, b = b, a%b
        x0, x1 = x1 - q * x0, x0
    if x1 < 0: x1 += b0
    return x1
 
if __name__ == '__main__':
    n = [7,8,109,551753]
    a = [7,0,3,46552]
    print (chinese_remainder(n, a))

NameError: name 'reduce' is not defined